In [17]:
# ------------------------- init requirements -------------
from typing import Tuple
from ipywidgets      import widgets,interact,interactive
from IPython.display import display
import pandas as pd
import os
from pathlib import Path
import plotly.graph_objects as go
from PIL import Image
'''
csv data format

    "game starttime",
    "game endtime",
    "gameId",                counter
    "offense_team_id",       when event=goal|shot     team offenser 
    "offense_team_name",     when event=goal|shot     team name
    "offense_team_tricode",  when event=goal|shot     team tricode
    "goal",                  goal = 1 | shot= 0
    "x_coords",              
    "y_coords",
    "goalie_id",             goalkeeper 
    "goalie_name",           goalkeeper
    "shooter_id",
    "shooter_name",
    "shot type",
    "empty net",
    "strength_shorthand",
    "strength_even",
    "strength_powerplay",
'''
full_path = os.getcwd()
root_path = str(Path(full_path).parents[0])

#define initial parameters
im      = Image.open(r'{r}/figures/nhl_rink.png'.format(r=root_path)) 
im2     = Image.open(r'{r}/figures/nhl_rink_top.png'.format(r=root_path)) 
seasons = ['20162017','20172018','20182019','20192020', '20202021']
teams   = ['NOP','NJD','NYI','NYR','PHI','PIT','BOS','BUF','MTL','OTT','TOR','FLA','WSH','CHI','DET','NSH','STL','CGY','COL','EDM','VAN','ANA','DAL','LAK','SJS','CBJ','MIN','WPG','ARI','VGK','SEA']

print('csv on ',root_path+"/ift6758/data/tabular") 
#csv on  /home/williamglazer/Documents/school/DS/IFT6758/ift6758/data/tabular

csv on  /Users/olivercordoba/Documents/github/IFT6758/IFT6758/ift6758/data/tabular


In [18]:
# fix to inline rendering issue
# displays plots in a separate tab within the default broswer
# https://stackoverflow.com/questions/50250010/plotly-chart-is-not-displayed-in-pycharm

import plotly.io as pio
pio.renderers.default = 'browser'

In [19]:
'''
 based on season and team, draw all shooting point on the map
 
'''
def vgraph(season,team):
    
    # Create figure
    fig = go.Figure()
    
    print('season: {s} team: {t} '.format(s=season,t=team))
    
    #load data for selected season / team
    csv_path  = root_path+'/ift6758/data/tabular/'+season+'.csv'
    df = pd.read_csv(csv_path)
    df2 =df[df['offense_team_tricode'] == team]
    
    # Add traces
    fig.add_trace(
        go.Scatter(
            x=df2.loc[:, 'x_coords'],
            y=df2.loc[:, 'y_coords'],
            mode="markers",
            marker=dict(color="DarkOrange"),
        )
    )

   
    # Add background image
    fig.add_layout_image(
        dict(
            source=im,
            xref="x",
            yref="y",
            x=-100,   # dims in feet
            y=42.5,
            sizex=200,
            sizey=85,
            sizing="stretch",
            opacity=0.8,
            layer="below"
        )
    )
    
    
    #run view
    fig.update_layout(
        template="plotly_white"
    )
    fig.show()
    
    
w = interactive(vgraph,  season=widgets.Dropdown(options=seasons, value='20162017', description='Season',),
                         team  =widgets.Dropdown(options=teams,   value='MTL',      description='Team'),
                       )


display(w)
    

interactive(children=(Dropdown(description='Season', options=('20162017', '20172018', '20182019', '20192020', …

In [69]:
''' 
 transpose cordinate Y
 We will assume that all shots were realized from the opponent side. 
 but a more precise results will require to know the rink side at time
 of the shot for current team
'''
def transposeX(xval, yval):
    if xval < 0 :
        xval = xval*(-1)
    xval -= 89.0
    return xval*(-1)

def transposeY(xval, yval):
    if xval < 0 :
        yval = yval*(-1)
    return yval

        

In [70]:
'''
 based on season and team, draw all shooting point on the map
 
'''
def vgraph(season,team):
    
    # Create figure
    fig = go.Figure()
    
    print('season: {s} team: {t} '.format(s=season,t=team))
    
    # load data for selected season / team
    csv_path  = root_path+'/ift6758/data/tabular/'+season+'.csv'
    df = pd.read_csv(csv_path)
    
    # league shoot count , create unique key
    df['team_game'] = df.apply(lambda row: str(row['game starttime'])+str(row['offense_team_tricode']), axis=1)
    l_shoot_count  =  df.shape[0] 
    
    # all teams season games counter
    season_games_counter = df['team_game'].unique().shape[0]
    
    # Filter data leaving only current team
    df_team = df[df['offense_team_tricode'] == team].copy()
    
    # total shots on current game is equal at number of records on this date for this team 
    team_season_games = df_team['game starttime'].unique().shape[0]    
    
    '''  start funny thing ''' 
    # ----------------------------------------------------------------------------
    # build [excess_shot_rate] row, that represent on a tile size="tile_sz" 
    # the difference between the average shot on this tile for all teams all games
    # and the current number of shoots for the current team
    # ----------------------------------------------------------------------------
    
    tile_sz = 5 
    x_range = list(range(-100,100,tile_sz))
    y_range = list(range( -50, 50,tile_sz))
    
    for xval in x_range:
        for yval in y_range:
            # all shoot inside the current tile
            tile_shots = df.loc[(df['x_coords'] >= xval         ) & 
                                (df['x_coords'] < (xval+tile_sz)) & 
                                (df['y_coords'] >= yval         ) &
                                (df['y_coords'] < (yval+tile_sz)) ]
            tile_count = tile_shots.shape[0]
            tile_mean_per_game  = tile_count/season_games_counter
            # team shoot inside current tile
            teamtile_shoots = df_team.loc[(df_team['x_coords'] >= xval         ) & 
                                          (df_team['x_coords'] < (xval+tile_sz)) & 
                                          (df_team['y_coords'] >= yval         ) &
                                          (df_team['y_coords'] < (yval+tile_sz)) ]
            tg_shoots = teamtile_shoots.shape[0]
            tile_mean_per_game_per_team = tg_shoots/team_season_games
            #find all records that match this coordinates to include the value 
            for idx in teamtile_shoots.index:
                df_team.loc[idx,'excess_shot_rate'] = tile_mean_per_game_per_team -  tile_mean_per_game 
                
    '''  ends funny thing ''' 
    # Transpose X and Y based on current localization  
    df_team['y_abs'] = df_team.apply(lambda row: transposeX(row.x_coords,row.y_coords),axis = 1)
    df_team['x_abs'] = df_team.apply(lambda row: transposeY(row.x_coords,row.y_coords),axis = 1)

    #df_team['y_abs'] = df_team.apply(lambda row: (abs(row.x_coords)-89.0)*(-1.0), axis = 1)
    #df_team['x_abs'] = df_team.apply(lambda row: (row.y_coords*(-1.0)),      axis = 1)
    #compute shots on current game
    # df3 = df_team.groupby(['game starttime'])['game starttime'].count().to_frame()
    # df3.loc[row['game starttime'],:]['game starttime']
    
    # Trace test 01 : scatter, dont use escess shot rate
    '''
    fig.add_trace(
        go.Scatter(
            x=df_team.loc[:, 'y_coords'],
            y=df_team.loc[:, 'y_abs'],
            mode="markers",
            marker=dict(color="DarkOrange"),
        )
    )
    '''
    # Trace test 02 : HeatMap
    '''
        fig.add_trace(
        go.Heatmap(
            z=df_team.loc[:, 'excess_shot_rate'],
            x=df_team.loc[:, 'y_coords'],
            y=df_team.loc[:, 'y_abs'],
            opacity=0.6,
        colorscale='Viridis'
        )
    )
    ''' 
    colorscale = [[0, 'Blue'], [0.5, 'white'], [1, 'Red']]
    # Trace test 03 : Contour
    
    fig.add_trace(
        go.Contour(
            z=df_team.loc[:, 'excess_shot_rate'],
            x=df_team.loc[:, 'x_abs'],
            y=df_team.loc[:, 'y_abs'],
            opacity=0.6,
            colorscale=colorscale
        #colorscale='Viridis'
        )
    )
    
   
    # Add background image with the game limits
    fig.add_layout_image(
        dict(
            source=im2,
            xref="x",
            yref="y",
            x=-42.5,   # dims in feet
            y=-11,
            sizex=85,
            sizey=100,
            sizing="stretch",
            layer="below"
        )
    )
    
    # reverse Y axis to be top-down direction
    fig.update_yaxes(
        autorange="reversed"
    )
    
    # header title -------------------------- 
    fig.add_annotation(x=.5, y=-22,
            text="Unblocked Shot Rate",
            showarrow=False,
            font=dict(
                family="verdana, Arial",
                size=18, 
                color="#333333",
            )
    )
    
    # axes, scales and margins --------------
    fig.update_layout(
        showlegend=False,
        xaxis_title="Distace from centre of rink (ft)",
        yaxis_title="Distance from goal line (ft)",
        font=dict(
            family="Courier New, monospace",
            size=12,
            color="#333333"
        ),
        width=85*8,
        height=100*8,
        autosize=True,
        margin=dict(t=50, b=0, l=50, r=0), # graph margins
        template="plotly_white"
    )

    # run the visualization    
    fig.show()
    
    
    
w = interactive(vgraph,  season=widgets.Dropdown(options=seasons, value='20162017', description='Season',),
                         team  =widgets.Dropdown(options=teams,   value='MTL',      description='Team'),
                       )


display(w)

interactive(children=(Dropdown(description='Season', options=('20162017', '20172018', '20182019', '20192020', …

In [72]:
csv_path  = root_path+'/ift6758/data/tabular/'+'20162017'+'.csv'
df = pd.read_csv(csv_path)
df.head()
print(df['offense_team_tricode'].unique())

['TOR' 'OTT' 'STL' 'CHI' 'EDM' 'CGY' 'LAK' 'SJS' 'MTL' 'BUF' 'NYR' 'NYI'
 'BOS' 'CBJ' 'DET' 'TBL' 'NJD' 'FLA' 'WSH' 'PIT' 'MIN' 'CAR' 'WPG' 'ANA'
 'DAL' 'NSH' 'PHI' 'COL' 'ARI' 'VAN']
